In [1]:
import re, os, string, warnings, json
warnings.filterwarnings("ignore")
import math
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt', quiet=True)
from collections import Counter
from rapidfuzz import fuzz
from word2number import w2n
import dateparser
from dateutil import parser as duparser

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("synthetic_dataset.csv", encoding_errors="replace")

In [3]:
# This is being done so as to confirm to a same schema for data throughout the project
# Heuristic rename if your headers differ slightly
rename_map = {
    "reference": "gold_reference",
    "gold": "gold_reference",
    "answer": "ai_response",
    "response": "ai_response",
}
for k,v in rename_map.items():
    if k in df.columns and v not in df.columns:
        df.rename(columns={k:v}, inplace=True)

# Ensure exactly the 4 we use exist
for c in ["question","context","gold_reference","ai_response"]:
    if c not in df.columns: df[c] = ""

def _clean(s): 
    return (str(s) if pd.notna(s) else "").strip()

UNANSWERABLE_MARKERS = {
    "unanswerable","not answerable","cannot be determined","insufficient context",
    "not available in passage","unknown","na","n/a"
}

def derive_flags(row):
    gold = _clean(row["gold_reference"]).lower()
    ctx  = _clean(row["context"])
    is_unanswerable = gold in UNANSWERABLE_MARKERS
    context_missing = (ctx == "")
    no_gold = (gold == "")  # empty gold means judge vs context only
    return pd.Series({
        "is_unanswerable": is_unanswerable,
        "context_missing": context_missing,
        "no_gold": no_gold
    })

flags = df.apply(derive_flags, axis=1)
df = pd.concat([df, flags], axis=1)
df.head(3)

question  \
0  What did the Court of Justice not acknowledge?   
1  What did the Court of Justice not acknowledge?   
2  What did the Court of Justice not acknowledge?   

                                             context gold_reference  \
0  Fourth, national courts have a duty to interpr...            NaN   
1  Fourth, national courts have a duty to interpr...            NaN   
2  Fourth, national courts have a duty to interpr...            NaN   

                                         ai_response  is_unanswerable  \
0                    The passage doesn’t state this.            False   
1  It states that what did the court of justice w...            False   
2  The decision was influenced by the Paris Conve...            False   

   context_missing  no_gold  
0            False     True  
1            False     True  
2            False     True

In [4]:
# ---------- custom functions for various use cases ----------
def to_text(x) -> str:
    if x is None:
        return ""
    try:
        import pandas as _pd
        if _pd.isna(x):
            return ""
    except Exception:
        pass
    return str(x)

def sent_split(raw: str):
    raw = to_text(raw)
    parts = re.split(r"[\.?!;\n]+", raw)
    return [p.strip() for p in parts if p and p.strip()]

In [ ]:
# Coherence scoring
# based on factors like repetition of words, fluency, contradiction, similarity scores of adjacent sentences, use of punctuation in the end.

def coherence_subscores(resp_raw: str) -> dict:
    resp = to_text(resp_raw).strip()
    if not resp:
        return {
            "repetition": 0.0,
            "fluency": 0.0,
            "contradiction": 0.0,
            "flow": 0.0,
            "completeness": 0.0
        }

    # (1) Repetition penalty (bigrams/trigrams)
    tokens = resp.split()
    n = len(tokens)
    bigrams = [" ".join(tokens[i:i+2]) for i in range(n-1)]
    trigrams = [" ".join(tokens[i:i+3]) for i in range(n-2)]
    rep_ratio = 0.0
    if bigrams or trigrams:
        uniq = len(set(bigrams+trigrams))
        total = len(bigrams+trigrams) or 1
        rep_ratio = 1.0 - (uniq/total)   # higher = more repetition
    repetition = 1.0 - min(rep_ratio,1.0)  # invert: 1=good (low repetition)

    # (2) Fluency proxy
    punct_ratio = len(re.findall(r"[^\w\s]", resp)) / max(len(resp),1)
    avg_len = np.mean([len(w) for w in tokens]) if tokens else 0
    fluency = 1.0
    if punct_ratio > 0.15: fluency -= 0.3
    if avg_len < 2: fluency -= 0.3
    if avg_len > 12: fluency -= 0.3
    fluency = max(0.0, fluency)

    # (3) Contradiction proxy (look for "not"/"never" before repeated entity/number)
    ents = set(re.findall(r"\b[A-Z][a-z]+\b", resp))
    nums = set(re.findall(r"\d+(?:\.\d+)?", resp))
    contradictions = 0
    for e in ents|nums:
        hits = re.findall(rf"(not|never)\s+\b{re.escape(e)}\b", resp, re.I)
        if len(hits) > 0 and resp.lower().count(e.lower()) > 1:
            contradictions += 1
    contradiction = 1.0 - min(contradictions,3)/3.0  # scale down to 0

    # (4) Flow: adjacent sentence similarity
    sents = sent_split(resp)
    flow = 1.0
    if len(sents) >= 2:
        vect = TfidfVectorizer(min_df=1, ngram_range=(1,2))
        try:
            X = vect.fit_transform(sents)
            sims = cosine_similarity(X)
            adj = []
            for i in range(len(sents)-1):
                adj.append(sims[i,i+1])
            mean_sim = np.mean(adj) if adj else 0.0
            flow = float(np.clip(mean_sim,0.0,1.0))
        except ValueError:
            flow = 0.0

    # (5) Completeness: ends with .,? or ! → good
    completeness = 1.0 if resp[-1] in ".?!" else 0.5

    return {
        "repetition": repetition,
        "fluency": fluency,
        "contradiction": contradiction,
        "flow": flow,
        "completeness": completeness
    }


# Final coherence score

def coherence_score_row(row: pd.Series, weights=(0.25,0.2,0.15,0.25,0.15), tau_label=0.75):
    resp = to_text(row.get("ai_response",""))
    subs = coherence_subscores(resp)

    w_rep,w_flu,w_con,w_flow,w_comp = weights
    score = (w_rep*subs["repetition"] + w_flu*subs["fluency"] +
             w_con*subs["contradiction"] + w_flow*subs["flow"] +
             w_comp*subs["completeness"])
    score = float(min(1.0,max(0.0,score)))
    label = int(score >= tau_label)
    return {"coher_score": score, "coher_label": label, **subs}

def add_coherence_scores(df: pd.DataFrame,
                         weights=(0.25,0.2,0.15,0.25,0.15),
                         tau_label=0.75) -> pd.DataFrame:
    rows = [coherence_score_row(r, weights=weights, tau_label=tau_label) for _, r in df.iterrows()]
    return pd.concat([df.reset_index(drop=True), pd.DataFrame(rows)], axis=1)

In [8]:
df = add_coherence_scores(df, tau_label=0.75)
df[["ai_response","coher_score","coher_label","repetition","fluency","flow","completeness"]].head()

ai_response  coher_score  \
0                    The passage doesn’t state this.     1.000000   
1  It states that what did the court of justice w...     1.000000   
2  The decision was influenced by the Paris Conve...     1.000000   
3                               No answer in passage     0.925000   
4  Bullet 1) The Court accepted horizontal effect...     0.759396   

   coher_label  repetition  fluency      flow  completeness  
0            1         1.0      1.0  1.000000           1.0  
1            1         1.0      1.0  1.000000           1.0  
2            1         1.0      1.0  1.000000           1.0  
3            1         1.0      1.0  1.000000           0.5  
4            1         1.0      1.0  0.037585           1.0

In [9]:
df.to_csv("scored datasets/coherence_scores_added.csv")